In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cvxpy as cp

In [4]:
sns.set_theme(context="notebook")

## Import data

In [5]:
def get_demand_mask_and_price(df, colName, dt):
    col = df[colName].to_numpy() / dt
    mask = col > 0
    price = col[mask][0]
    return mask, price

In [6]:
##### loads
df = pd.read_csv("site1_load.csv")
dt = 1/4 # in hours
powerLoad, heatLoad = df['Power Load [kWh]'].to_numpy(), df['Heat Load [kWh]'].to_numpy()

##### prices
df = pd.read_csv("power_price_B20.csv")
energyPricePower = df["energyPrice"].to_numpy()
# from per kW to per kWh in 15 minutes
powerDemand = [get_demand_mask_and_price(df, colName, dt) for colName in ["peakDemandSummerPrice",
                                                                          "partialPeakDemandSummerPrice",
                                                                          "demandSummerPrice",
                                                                          "peakDemandWinterPrice",
                                                                          "demandWinterPrice"]]
df = pd.read_csv("gas_price.csv")
energyPriceGas = df["energyPrice"].to_numpy()

##### emissions
df = pd.read_csv("power_grid_emissions.csv")
df.ffill(inplace=True)
emissionsPower = df["MOER version 2.0"].to_numpy()
df = pd.read_csv("gas_emissions.csv")
emissionsGas = df["gasEmissions"].to_numpy()

# clear memory
del df

In [7]:
n = len(powerLoad)

## Baseline

We only have a natural gas furnace to supply heat, and no storage.

In [11]:
##### variables and parameters
# financial
CRF = cp.Parameter(nonneg=True) # capital recovery factor TBD
# gas furnace
gasInputGF = cp.Variable(n, nonneg=True)
capaPriceGF = cp.Parameter(nonneg=True) # $/kW TBD
effGF = cp.Parameter(nonneg=True) # %

##### useful quantities
# gas furnace
heatOutputGF = effGF*gasInputGF
capaGF = cp.max(heatOutputGF / dt)
# consumption
gasConsumption = gasInputGF
powerConsumption = powerLoad + 0 # no heat pump or batteries here
# loads
heatSupply = heatOutputGF
# costs
opexPower = energyPricePower@powerConsumption + np.sum([cp.max(powerConsumption[d[0]])*d[1] for d in powerDemand])
opexGas = energyPriceGas@gasConsumption
opex = opexPower + opexGas
capexGF = capaGF*capaPriceGF
capex = capexGF
# emissions
emissions = emissionsPower@powerConsumption + emissionsGas@gasConsumption

##### constraints
cons = []
# meet load
cons += [heatSupply == heatLoad]                                  

##### objective function
obj = cp.Minimize(opex + CRF*capex)

#### solve problem
CRF.value = 1/20 # % TBD
capaPriceGF.value =  200 # $/kW TBD
effGF.value = 0.85 # %
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.CLARABEL)
print(prob.status)
print('cost = ', np.round(obj.value), "$ per year")
print('emissions = ', np.round(emissions.value), "kgCO2 per year")
print(' Gas Furnace capacity= ', np.round(capaGF.value), "kW")
# TODO LCOS and LCOE, same for emissions

baseline_cost = obj.value
baseline_emissions = emissions.value

/Users/aramis/miniconda3/envs/spcqe-dev/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:226: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(DPP_ERROR_MSG)


optimal
cost =  5674230.0 $ per year
emissions =  16043137.0 kgCO2 per year
 Gas Furnace capacity=  35748.0 kW


## With a heat pump

Natural gas furnace and heat pump

In [20]:
##### variables and parameters
# financial
CRF = cp.Parameter(nonneg=True) # capital recovery factor TBD
# gas furnace
gasInputGF = cp.Variable(n, nonneg=True)
effGF = cp.Parameter(nonneg=True) # %
capaPriceGF = cp.Parameter(nonneg=True) # $/kW TBD
# heat pump
powerInputHP = cp.Variable(n, nonneg=True)
copHP = cp.Parameter(nonneg=True) # % TBD
capaPriceHP = cp.Parameter(nonneg=True) # $/kW TBD
#should add ramp rates maybe ?

##### useful quantities
# gas furnace
heatOutputGF = effGF*gasInputGF
capaGF = cp.max(heatOutputGF / dt)
# heat pump
heatOutputHP = copHP*powerInputHP
capaHP = cp.max(heatOutputHP / dt)
# consumption
gasConsumption = gasInputGF
powerConsumption = powerLoad + powerInputHP
# loads
heatSupply = heatOutputGF + heatOutputHP
# costs
opexPower = energyPricePower@powerConsumption + np.sum([cp.max(powerConsumption[d[0]])*d[1] for d in powerDemand])
opexGas = energyPriceGas@gasConsumption
opex = opexPower + opexGas
capexGF = capaGF*capaPriceGF
capexHP = capaHP*capaPriceHP
capex = capexGF + capexHP
# emissions
emissions = emissionsPower@powerConsumption + emissionsGas@gasConsumption

##### constraints
cons = []
# meet load
cons += [heatSupply == heatLoad]                                  
# reduce emissions
cons += [emissions <= 0.7*baseline_emissions] # 30% reduction

##### objective function
obj = cp.Minimize(opex + CRF*capex)

#### solve problem
CRF.value = 1/20 # % TBD
capaPriceGF.value =  200 # $/kW TBD
effGF.value = 0.85 # %
copHP.value = 3 # TBD
capaPriceHP.value =  1000 # $/kW TBD
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.CLARABEL)
print(prob.status)
print('cost = ', np.round(obj.value), "$ per year", "(", np.round(100*(obj.value-baseline_cost)/baseline_cost, 2), "% increase relative to baseline)")
print('emissions = ', np.round(emissions.value), "kgCO2 per year", "(", -np.round(100*(emissions.value-baseline_emissions)/baseline_emissions, 2), "% reduction relative to baseline)")
print('gas furnace capacity= ', np.round(capaGF.value), "kW")
print('heat pump capacity= ', np.round(capaHP.value), "kW")
# TODO LCOS and LCOE, same for emissions

optimal
cost =  5966389.0 $ per year ( 5.15 % increase relative to baseline)
emissions =  11230196.0 kgCO2 per year ( 30.0 % reduction relative to baseline)
gas furnace capacity=  27207.0 kW
heat pump capacity=  8541.0 kW
